<a href="https://colab.research.google.com/github/AyishaR/Spokendigit/blob/master/Spokendigit_Five_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
from torch.utils.data import Dataset, random_split, DataLoader, TensorDataset
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import tarfile
import os
import librosa
import pandas as pd
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import librosa.display
import sklearn
import matplotlib
import csv
from PIL import Image
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score

#Data

In [4]:
digit = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']

#Dataset 



Spoken_mnist_finalfts.csv is generated in 'Spokenmnist feature extraction.ipynb'.

In [9]:
finalfts = pd.read_csv("Spoken_digit_finalfts.csv")

In [6]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
finalfts[finalfts.columns[1:]] = scale.fit_transform(finalfts[finalfts.columns[1:]])

In [10]:
spokendset = TensorDataset(torch.tensor(np.array(finalfts[finalfts.columns[1:]].astype('float32'))),torch.tensor(finalfts['Label'])) 

In [11]:
# 90-10 split
size = len(spokendset)
val_size = int(0.1 * size)
train_size = size - val_size 

train_dset, val_dset = random_split(spokendset, [train_size, val_size])

train_size, val_size

(21298, 2366)

In [12]:
train_dl = DataLoader(train_dset, 512, True)
val_dl = DataLoader(val_dset, 512)

# Device


In [13]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

In [14]:
device = get_default_device()
device

device(type='cpu')

In [15]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)

#Train

In [16]:
class SpokenDigitModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(173, 1024)
        self.l2 = nn.Linear(1024, 512)
        self.l3 = nn.Linear(512, 64)
        self.l4 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = self.l4(x)
        return x

    def training_step(self, batch):
        inputs, labels = batch
        outputs = self(inputs)
        loss = F.cross_entropy(outputs, labels)
        return loss

    def validation_step(self, batch):
        inputs, labels = batch
        outputs = self(inputs)
        loss = F.cross_entropy(outputs, labels)
        _, pred = torch.max(outputs, 1)
        accuracy = torch.tensor(torch.sum(pred==labels).item()/len(pred))
        return [loss.detach(), accuracy.detach()] 

In [17]:
def evaluate(model, loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in tqdm(loader)]
    outputs = torch.tensor(outputs).T
    loss, accuracy = torch.mean(outputs, dim=1)
    return {"loss" : loss.item(), "accuracy" : accuracy.item()}

In [18]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [19]:
def fit(model, train_loader, val_loader, epochs, lr, optimizer_function = torch.optim.Adam):
    history = []
    optimizer = optimizer_function(model.parameters(), lr)
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, lr, epochs=epochs, steps_per_epoch=len(train_loader))
    for epoch in range(epochs):
        print("Epoch ", epoch)
        #Train
        model.train()
        lrs = []
        tr_loss = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            tr_loss.append(loss)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()
            lrs.append(get_lr(optimizer))
            sched.step()
        #Validate
        result = evaluate(model, val_loader)
        result["lrs"] = lrs
        result["train loss"] = torch.stack(tr_loss).mean().item()
 
        print("Last lr: ", lrs[-1]," Train_loss: ", result["train loss"], " Val_loss: ", result['loss'], " Accuracy: ", result['accuracy'])
        history.append(result)         
    return history

In [20]:
model = to_device(SpokenDigitModel(), device)
history = []
evaluate(model, val_dl)

{'accuracy': 0.11331318318843842, 'loss': 2.9422481060028076}

In [22]:
history.append(fit(model, train_dl, val_dl, 64, 0.01))

Epoch  0



Last lr:  0.00046125331558460707  Train_loss:  2.0962071418762207  Val_loss:  1.7959665060043335  Accuracy:  0.3839131295681
Epoch  1



Last lr:  0.000649371685235026  Train_loss:  1.6721926927566528  Val_loss:  1.5381710529327393  Accuracy:  0.4640895426273346
Epoch  2



Last lr:  0.0009593538973555506  Train_loss:  1.477335810661316  Val_loss:  1.4922049045562744  Accuracy:  0.4857262074947357
Epoch  3



Last lr:  0.0013828987634391283  Train_loss:  1.4045330286026  Val_loss:  1.3956390619277954  Accuracy:  0.5016951560974121
Epoch  4


...


Last lr:  0.00019545260862094147  Train_loss:  0.05168016627430916  Val_loss:  2.2133004665374756  Accuracy:  0.7537834048271179
Epoch  60



Last lr:  0.00011027567791908655  Train_loss:  0.05067480728030205  Val_loss:  2.2165989875793457  Accuracy:  0.7553459405899048
Epoch  61



Last lr:  4.913417694027851e-05  Train_loss:  0.05024070292711258  Val_loss:  2.2178587913513184  Accuracy:  0.7569084167480469
Epoch  62



Last lr:  1.232864537599174e-05  Train_loss:  0.05011792480945587  Val_loss:  2.2188334465026855  Accuracy:  0.7554982304573059
Epoch  63



Last lr:  4e-08  Train_loss:  0.04999082535505295  Val_loss:  2.2191309928894043  Accuracy:  0.7554982304573059


In [23]:
@torch.no_grad()
def predict_dl(model, dl):
    torch.cuda.empty_cache()
    batch_probs = []
    batch_targ = []
    for xb, yb in tqdm(dl):
        probs = model(xb)
        batch_probs.append(probs.cpu().detach())
        batch_targ.append(yb.cpu().detach())
    batch_probs = torch.cat(batch_probs)
    batch_targ = torch.cat(batch_targ)
    return [list(values).index(max(values)) for values in batch_probs], batch_targ

In [26]:
r = evaluate(model, val_dl)
yp, yt = predict_dl(model, val_dl)
print("Loss: ", r['loss'], "\nAccuracy: ", r['accuracy'], "\nF-score: ", f1_score(yt, yp, average='micro'))


Loss:  2.2191309928894043 
Accuracy:  0.7554982304573059 
F-score:  0.7540152155536771


In [27]:
torch.save(model, '/content/spokendigit_lr_all.pth')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type SpokenDigitModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
